In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:

path = r' #copy path here
data = pd.read_csv(path)

In [20]:
df = data.copy()

In [21]:
def cleaning_data (df, date_begin, day_window = 3, **args):
    #format date 
    df['act_date'] = pd.to_datetime(df['act_date'])
    df['day0_date'] = pd.to_datetime(df['day0_date'])
    #remove missing and useless values
    ind = df[df.day_diff < 0].index
    df.drop(ind, axis= 0, inplace=True)
    #query data
    day_range = pd.date_range(start = date_begin, periods=day_window, freq='1D' )
    df.query('day0_date in @day_range', inplace = True)
    df = df[df.country == args ['country']]
    df = df[df.media_source == args['media_source']]
    df = df[df.app_version == args['app_version']]
    return df    

    

In [22]:
df_pivot = cleaning_data(df, date_begin = '2021-02-20', country = 'United States', media_source = 'Organic', app_version = '1.0.4')

In [27]:
def game_info_simple(df_pivot, date_begin = '2021-02-20', day_window = 3):
    df_pivot = df_pivot.groupby('day_diff').agg({'dau':np.sum, 'time_in_game_sum':np.mean, 'rv_imp_sum':np.mean, 'is_imp_sum':np.mean, 'battle_end_sum':np.mean, 'battle_play_sum':np.mean}).loc[:day_window]
    df_pivot['dau'] = df_pivot['dau']/df_pivot.iloc[0,0]
    #add grand total row
    df_pivot.loc['Grand Total'] = df_pivot.sum(axis = 0)
    return df_pivot


In [28]:
game_info_simple(df_pivot)

,dau,time_in_game_sum,rv_imp_sum,is_imp_sum,battle_end_sum,battle_play_sum
day_diff,,,,,,
0,1.000000,299.885891,0.913449,1.719041,8.179760,8.972037
1,0.137150,404.228583,0.741935,1.722581,6.296774,7.006452
2,0.061252,244.447326,0.240000,0.773333,3.573333,4.213333
3,0.047936,271.743861,0.545455,0.800000,4.654545,5.327273
Grand Total,1.246338,1220.305661,2.440839,5.014955,22.704413,25.519095
